In [70]:
import pandas as pd
import numpy as np
import sqlite3 
import csv

In [107]:
df = pd.read_csv('data.csv')
con = sqlite3.connect("covid.db")
df.to_sql('covid', conn, if_exists='append', index = False)
pd.set_option('display.max_columns', None)
df.tail()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,new_vaccinations_smoothed_per_million,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
86413,ZWE,Africa,Zimbabwe,2021-05-02,38281.0,21.0,27.857,1570.0,2.0,1.857,2575.603,1.413,1.874,105.632,0.135,0.125,0.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1133.0,495983.0,33.37,0.076,1862.0,0.125,0.015,66.8,tests performed,524199.0,430068.0,94131.0,3900.0,19169.0,3.53,2.89,0.63,1290.0,NaN,14862927.0,42.729,19.6,2.822,1.882,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571
86414,ZWE,Africa,Zimbabwe,2021-05-03,38293.0,12.0,27.286,1573.0,3.0,1.857,2576.410,0.807,1.836,105.834,0.202,0.125,0.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,537516.0,437751.0,99765.0,13317.0,17987.0,3.62,2.95,0.67,1210.0,NaN,14862927.0,42.729,19.6,2.822,1.882,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571
86415,ZWE,Africa,Zimbabwe,2021-05-04,38327.0,34.0,23.286,1574.0,1.0,1.286,2578.698,2.288,1.567,105.901,0.067,0.087,0.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,559777.0,452191.0,107586.0,22261.0,17977.0,3.77,3.04,0.72,1210.0,NaN,14862927.0,42.729,19.6,2.822,1.882,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571
86416,ZWE,Africa,Zimbabwe,2021-05-05,38357.0,30.0,23.714,1574.0,0.0,1.286,2580.716,2.018,1.596,105.901,0.000,0.087,0.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14862927.0,42.729,19.6,2.822,1.882,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571
86417,ZWE,Africa,Zimbabwe,2021-05-06,38398.0,41.0,23.286,1575.0,1.0,1.143,2583.475,2.759,1.567,105.968,0.067,0.077,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14862927.0,42.729,19.6,2.822,1.882,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571


How many countries are included in our data set?

In [103]:
countries = pd.read_sql_query("SELECT COUNT(DISTINCT location) FROM covid", con)
countries

,COUNT(DISTINCT location)
0,219


When was the most recent entry?

In [134]:
last_entry = pd.read_sql_query("SELECT date FROM covid ORDER BY date DESC LIMIT 1", con)
last_entry

,date
0,2021-05-06


What country had the highest single day deaths per million? 

In [133]:
max_daily_death_per_country = pd.read_sql_query("SELECT location, max(new_deaths_per_million), date FROM covid WHERE location != 'World' and location != 'Europe' and location != 'North America' and location != 'South America' and location != 'Asia' and location != 'European Union'", con)
max_daily_death_per_country

,location,max(new_deaths_per_million),date
0,Ecuador,218.329,2020-09-07


Select any country. How many times have the deaths per million exceeded 1,000 in one day?


In [132]:
death_per_country = pd.read_sql_query("SELECT location, COUNT(new_deaths_per_million) FROM covid WHERE new_deaths_per_million > 1.000 GROUP BY location", con)
death_per_country


,location,COUNT(new_deaths_per_million)
0,Afghanistan,34
1,Albania,4726
2,Andorra,1462
3,Antigua and Barbuda,408
4,Argentina,5015
...,...,...
145,Vanuatu,17
146,World,2822
147,Yemen,51
148,Zambia,153


Extract the deaths per million for US, France, and India. Plot the time series data and compare results to what the media is currently saying

In [142]:
us_france_india = pd.read_sql_query("SELECT date, location, new_deaths_per_million FROM covid WHERE location='India' UNION SELECT date, location, new_deaths_per_million FROM covid WHERE location='France' UNION SELECT date, location, new_deaths_per_million FROM covid WHERE location='United States'; ", con)
us_france_india


,date,location,new_deaths_per_million
1398,2021-05-05,India,2.884
1399,2021-05-05,United States,2.344
1400,2021-05-06,France,3.214
1401,2021-05-06,India,0.000
1402,2021-05-06,United States,2.375


Plot the deaths per million and cases per million by continent for your birthday in 2020. What does the data say?